# Ontario Wildfire Intelligence Dashboard

**Objective:** This notebook builds and deploys an interactive dashboard to analyze wildfire data in Ontario, correlating it with weather and pollution (PM2.5) metrics.

**Workflow:**
1. **Data Pipeline:** Fetch, unzip, and clean the raw data.
2. **Setup:** Install dashboarding and tunneling libraries.
3. **App Development:** Write the `app.py` script containing the dashboard logic and visualizations.
4. **Deployment:** Launch the app via a public `ngrok` tunnel.

## 1. Data Pipeline (ETL)
We start by downloading the raw dataset from the source, extracting the contents, and performing basic cleaning (parsing dates, removing missing values) to create `final_dashboard_data.csv`.

In [6]:
import pandas as pd
import os
import zipfile

!wget -O X_Wildfire.zip https://github.com/MajdBa7r/AQI/raw/main/X_Wildfire.zip -q

with zipfile.ZipFile("X_Wildfire.zip", 'r') as zip_ref:
    zip_ref.extractall(".")

csv_files = [f for f in os.listdir(".") if f.endswith(".csv")]
df = pd.read_csv(csv_files[0])

df['date'] = pd.to_datetime(df['date'], errors='coerce')
df = df.dropna(subset=['date'])

df.to_csv("final_dashboard_data.csv", index=False)

## 2. Environment Setup
We install the required libraries:
* `streamlit`: For the web application framework.
* `plotly`: For interactive charts.
* `pyngrok`: To expose the local server to a public URL.

In [7]:
!pip install -q streamlit plotly pyngrok pandas


## 3. Dashboard Application Logic
We write the entire Streamlit application into a file named `app.py`. This script defines:
* **Configuration:** Page layout and color themes.
* **Data Loading:** Efficiently loading the cleaned CSV.
* **Visualizations:**
    * *Time Series:* PM2.5 trends over time.
    * *Correlations:* Wind speed vs. Smoke, and Dryness vs. Temperature.
    * *Analysis:* Fire causes (violin plot) and Wind patterns (radar chart).
    * *Map:* Geospatial distribution of pollution.

In [8]:
%%writefile app.py

import streamlit as st
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
from plotly.subplots import make_subplots

# CONFIG
st.set_page_config(
    page_title="Ontario Wildfire Intelligence",
    page_icon="🔥",
    layout="wide"
)

# Set consistent color theme
COLOR_THEME = {
    'fire': '#FF6B6B',
    'no_fire': '#4ECDC4',
    'pm25': '#FFD166',
    'temp': '#EF476F',
    'wind': '#118AB2',
    'dryness': '#06D6A0'
}

@st.cache_data
def load_data():
    df = pd.read_csv("final_dashboard_data.csv")
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    df = df.dropna(subset=['date'])
    return df

df = load_data()

# Sidebar filters

st.sidebar.header("🔍 Filters")

date_range = st.sidebar.date_input(
    "Date Range",
    (df['date'].min().date(), df['date'].max().date())
)

cities = df['city'].dropna().unique().tolist()
selected_cities = st.sidebar.multiselect(
    "City",
    cities,
    default=cities[:3] if len(cities) >= 3 else cities
)

mask = pd.Series(True, index=df.index)

if len(date_range) == 2:
    mask &= (
        (df['date'] >= pd.to_datetime(date_range[0])) &
        (df['date'] <= pd.to_datetime(date_range[1]))
    )

if selected_cities:
    mask &= df['city'].isin(selected_cities)

data = df[mask]

# Fire flag
data["Fire"] = (data["FIRE_FINAL_SIZE"] > 0).astype(int)


st.title("🔥 Ontario Wildfire Intelligence Dashboard")
st.caption(
    "Interactive analytics linking meteorological conditions, air pollution (PM2.5), "
    "and wildfire occurrence."
)

# KPIs
c1, c2, c3, c4 = st.columns(4)

c1.metric("Avg PM2.5", f"{data['pm25'].mean():.1f}")
c2.metric("Max PM2.5", f"{data['pm25'].max():.1f}")
c3.metric("Fire Days", int(data["Fire"].sum()))
c4.metric("Avg Temp (°C)", f"{data['temperature_2m_mean (°C)'].mean():.1f}")

# 1️⃣ PM2.5 TRENDS OVER TIME
st.subheader("📈 PM2.5 Trends Over Time")

daily = (
    data.groupby("date")
    .agg(pm25_mean=("pm25", "mean"), fire_days=("Fire", "max"))
    .reset_index()
)

daily["pm25_rolling_7"] = daily["pm25_mean"].rolling(7).mean()

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=daily["date"],
    y=daily["pm25_mean"],
    mode="lines",
    name="Daily PM2.5",
    opacity=0.4,
    line=dict(color=COLOR_THEME['pm25'])
))

fig.add_trace(go.Scatter(
    x=daily["date"],
    y=daily["pm25_rolling_7"],
    mode="lines",
    name="7-day Rolling Mean",
    line=dict(width=3, color=COLOR_THEME['temp'])
))

fig.update_layout(
    template="plotly_dark",
    height=400,
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)'
)
st.plotly_chart(fig, use_container_width=True)

# ROW 1: Wind vs Smoke + Dryness vs Temperature
st.subheader("💨 Wind & Dryness Analysis")
col1, col2 = st.columns(2)

with col1:
    # 2️⃣ WIND VS SMOKE (PHYSICAL AUDIT)
    fig1 = px.scatter(
        data,
        x="wind_speed_10m_mean (km/h)",
        y="pm25",
        color="Fire",
        color_discrete_map={1: COLOR_THEME['fire'], 0: COLOR_THEME['no_fire']},
        labels={"Fire": "Fire Day"},
        opacity=0.6,
        template="plotly_dark",
        title="Wind Speed vs PM2.5"
    )
    fig1.update_layout(
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)'
    )
    st.plotly_chart(fig1, use_container_width=True)
    st.caption(
        "Higher PM2.5 concentrations are more likely under low-to-moderate wind speeds, "
        "especially on fire days, indicating smoke accumulation rather than dispersion."
    )

with col2:
    # 3️⃣ DRYNESS & TEMPERATURE VS FIRE
    fig2 = px.scatter(
        data,
        x="vapour_pressure_deficit_max (kPa)",
        y="temperature_2m_mean (°C)",
        color="Fire",
        color_discrete_map={1: COLOR_THEME['fire'], 0: COLOR_THEME['no_fire']},
        opacity=0.6,
        template="plotly_dark",
        title="Atmospheric Dryness & Temperature"
    )
    fig2.update_layout(
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)'
    )
    st.plotly_chart(fig2, use_container_width=True)
    st.caption(
        "Fire days cluster under hot and dry atmospheric conditions, confirming the "
        "dominant role of vapour pressure deficit in wildfire occurrence."
    )

# ROW 2: Fire Cause Violin Plot + Wind Radar
st.subheader("🔥 Fire Causes & Wind Patterns")
col3, col4 = st.columns(2)

with col3:
    # 4️⃣ FIRE CAUSE VIOLIN PLOT
    fire_days = data[data["FIRE_FINAL_SIZE"] > 0].copy()
    fig3, ax = plt.subplots(figsize=(10, 6))

    if not fire_days.empty and 'FIRE_GENERAL_CAUSE_CODE' in fire_days.columns:
        # Define a color palette consistent with our theme
        cause_palette = {
            code: COLOR_THEME['fire'] if i % 2 == 0 else COLOR_THEME['temp']
            for i, code in enumerate(fire_days['FIRE_GENERAL_CAUSE_CODE'].unique())
        }

        sns.violinplot(
            data=fire_days,
            x='FIRE_GENERAL_CAUSE_CODE',
            y='FIRE_FINAL_SIZE',
            palette=cause_palette,
            inner="quartile",
            hue='FIRE_GENERAL_CAUSE_CODE',
            dodge=False,
            ax=ax
        )

        ax.set_title("Fire Size by Cause: Human vs. Natural", fontsize=14, color='white')
        ax.set_xlabel("Fire Cause Code", color='white')
        ax.set_ylabel("Fire Size (Hectares)", color='white')
        ax.set_yscale('log')
        ax.legend(title="Cause Code", bbox_to_anchor=(1.02, 1), loc='upper left', frameon=False)
        ax.grid(axis='y', linestyle='--', alpha=0.3)

        # Set dark background
        fig3.patch.set_facecolor('#0E1117')
        ax.set_facecolor('#0E1117')
        ax.tick_params(colors='white')
        ax.spines['bottom'].set_color('white')
        ax.spines['top'].set_color('white')
        ax.spines['left'].set_color('white')
        ax.spines['right'].set_color('white')

        st.pyplot(fig3, use_container_width=True)
        st.caption("Violin plot showing distribution of fire sizes by cause code (log scale)")
    else:
        st.info("No fire data available for selected filters")

with col4:
    # 5️⃣ WIND RADAR PLOT
    if 'wind_speed_10m_max (km/h)' in data.columns and 'winddirection_10m_dominant (°)' in data.columns:
        radar_data = data.copy()
        radar_data['pm25_capped'] = radar_data['pm25'].clip(upper=150)

        fig4 = px.scatter_polar(
            radar_data,
            r='wind_speed_10m_max (km/h)',
            theta='winddirection_10m_dominant (°)',
            color='pm25_capped',
            range_color=[0, 150],
            hover_data=['date', 'pm25', 'wind_speed_10m_max (km/h)'],
            color_continuous_scale='YlOrRd',
            template='plotly_dark',
            title='Wind Direction & Smoke Concentration'
        )

        fig4.update_layout(
            polar=dict(
                bgcolor='rgb(10, 10, 10)',
                radialaxis=dict(
                    visible=True,
                    showticklabels=True,
                    tickfont=dict(color='white'),
                    gridcolor='gray'
                ),
                angularaxis=dict(
                    tickfont=dict(color='white'),
                    rotation=90,
                    direction="clockwise",
                    gridcolor='gray'
                )
            ),
            font=dict(color='white'),
            plot_bgcolor='rgba(0,0,0,0)',
            paper_bgcolor='rgba(0,0,0,0)'
        )

        st.plotly_chart(fig4, use_container_width=True)
        st.caption("Radar plot showing wind direction vs speed, colored by PM2.5 concentration")
    else:
        st.info("Wind data columns not available in the dataset")

# 6️⃣ MAP — FIRE & POLLUTION
st.subheader("🗺️ Fire & Pollution Spatial Pattern")

map_df = data.dropna(subset=["lat", "lon"])

if not map_df.empty:
    fig5 = px.scatter_mapbox(
        map_df,
        lat="lat",
        lon="lon",
        color="pm25",
        size="pm25",
        zoom=4,
        mapbox_style="carto-darkmatter",
        opacity=0.6,
        color_continuous_scale='YlOrRd',
        title="PM2.5 Concentration Map"
    )

    fig5.update_layout(
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)'
    )

    st.plotly_chart(fig5, use_container_width=True)
else:
    st.info("No geographical data available for selected filters")

# 7️⃣ DATA PREVIEW
st.subheader("📊 Data Snapshot")

cols_to_show = [
    "date", "city", "pm25",
    "temperature_2m_mean (°C)",
    "wind_speed_10m_mean (km/h)",
    "vapour_pressure_deficit_max (kPa)",
    "FIRE_FINAL_SIZE"
]

available_cols = [c for c in cols_to_show if c in data.columns]

st.dataframe(
    data[available_cols]
    .sort_values("date", ascending=False)
    .head(200),
    use_container_width=True
)

# FOOTER
st.markdown("---")
st.markdown(
    """
    **Dashboard Features:**
    - 📈 **PM2.5 Trends**: Time series analysis of air pollution
    - 💨 **Wind Analysis**: Relationship between wind patterns and smoke dispersion
    - 🔥 **Fire Risk Factors**: Temperature and dryness correlation with fires
    - 🗺️ **Spatial Patterns**: Geographic distribution of pollution and fires
    - 📊 **Cause Analysis**: Distribution of fire sizes by cause type
    - 🧭 **Wind Radar**: Directional analysis of wind and smoke patterns
    """
)

Overwriting app.py


## 4. Deployment & Execution
Since this notebook runs in a cloud environment (like Colab), we cannot access `localhost` directly. We use `ngrok` to create a secure tunnel that exposes port 8501 to a public URL, allowing you to view the dashboard in your browser.

In [9]:
from pyngrok import ngrok

ngrok.set_auth_token("37SsCSv2DNjzqmOAk3KyCauLco0_56dDQ4YDFVpMst9yuJmrz")
ngrok.kill()

public_url = ngrok.connect(8501)
print("Public URL:", public_url)

Public URL: NgrokTunnel: "https://aciform-stanley-derangedly.ngrok-free.dev" -> "http://localhost:8501"


In [ ]:
!streamlit run app.py


  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.231.30.235:8501

/content/app.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Fire"] = (data["FIRE_FINAL_SIZE"] > 0).astype(int)
2025-12-29 20:49:38.018 Please replace `use_container_width` with `width`.

`use_container_width` will be removed after 2025-12-31.

For `use_container_width=True`, use `width='stretch'`. For `use_container_width=False`, use `width='content'`.
2025-12-29 20:49:38.161 Please replace `use_container_width` with `width`.

`use_container_width` will be removed after 2025-12-31.

For `use_container_width=True`, use `width='stretch'`. For `use_container_width=Fa

## Acknowledgements
This project was developed with the assistance of:
* **AI Tools:** Used for code optimization, debugging, and structuring the dashboard logic.
* **YouTube Tutorials:** Referenced for best practices in Streamlit layout and `ngrok` deployment.